In [11]:
import pandas as pd
from lxml import html
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)


In [28]:
data_rt = pd.read_csv('news_texts.csv')

In [31]:
data_rt['content_norm'] = data_rt['content'].apply(normalize)

In [30]:
data_rt.dropna(inplace=True)

## Матричные разложения

Попробуем сначала матричные разложения. В SVD и в NMF одна из получаемых матриц имеет размерность (количество слов, количесто "тем"). Вектора из этих матриц и будут искомыми эбмедингами.

Для построение изнчальной матрицы слова на документы воспользуемся CountVectorizer из sklearn.

In [37]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X = cv.fit_transform(data_rt['content_norm'])

In [38]:
X.shape

(7212, 1000)

Разложим матрицу. Сначала попробуем только две размерности, чтобы визуализировать вектора.

In [85]:
svd = TruncatedSVD(50)
svd.fit(X)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)

In [46]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
X_tf = tfidf.fit_transform(data_rt['content_norm'])

In [86]:
svd_tf = TruncatedSVD(50)
svd_tf.fit(X_tf)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)

In [50]:
nmf = NMF(50)
nmf.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=50, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [51]:
nmf_tf = NMF(50)
nmf_tf.fit(X_tf)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=50, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [13]:
svd = TruncatedSVD(50)
svd.fit(X)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)

In [16]:
id2word = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}

In [17]:
id2vec_svd = nmf.components_.T

In [18]:
id2vec_nmf = svd.components_.T

In [58]:
fast_text = gensim.models.FastText([text.split() for text in data_rt['content_norm']], size=50, min_n=4, max_n=8)
w2v = gensim.models.Word2Vec([text.split() for text in data_rt['content_norm']], size=50, sg=1)


In [ ]:
fast_text = gensim.models.FastText([text.split() for text in data_rt['content_norm']], size=50, min_n=4, max_n=8)
w2v = gensim.models.Word2Vec([text.split() for text in data_rt['content_norm']], size=50, sg=1)


In [ ]:
corpus = [text.split() for text in data_rt['content'].apply(tokenize)]
fast_text = gensim.models.FastText(corpus, size=50, min_n=4, max_n=8)

### Векторные представления в настоящей задаче

Все вышеперечисленое относится к intrinsic (внутренним) метрикам. Есть также много других схожих (аналогии, корреляция с оценками людей и т.д). Но эти метрики не всегда показывают какой из методов сработает в реальной задаче. Поэтому при выборе методов и подборе параметров лучше ориентироваться на оценки качества решаемой задачи.

Проверим как все эти методы работают на задаче определения парафразов (предложений, которые выражают одно и то же, но не равны друг другу).

Данные взяты вот отсюда: http://paraphraser.ru/

Коллекция состоит из пар предложения (заголвков статей) и метки класса (-1,0,1). -1 не парафраз, 1 - парафраз, 0 - что-то непонятное.

In [26]:
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [27]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

Тексты короткие и их маловато, поэтому возьмем модели, обученные на новостных текстах.

Для решения задачи преобразуем каждый текст и конкатенируем их векторы.

### SVD

In [40]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)

In [87]:
X_text_1 = svd_tf.transform(tfidf.transform(data['text_1_norm']))
X_text_2 = svd_tf.transform(tfidf.transform(data['text_2_norm']))

X_text_tf = np.concatenate([X_text_1, X_text_2], axis=1)

In [88]:
X_text_tf.shape

(7227, 100)

In [89]:
X_text_1 = svd.transform(cv.transform(data['text_1_norm']))
X_text_2 = svd.transform(cv.transform(data['text_2_norm']))

X_text_cv = np.concatenate([X_text_1, X_text_2], axis=1)

In [90]:
X_text_cv.shape

(7227, 100)

In [44]:
y = data['label'].values
print(y.shape)

(7227,)


### NMF

Точно также (делаем пару векторов, конкатенируем, суём в логрег или рандом форест)

In [52]:
X_text_1_nmf_tf = nmf_tf.transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf_tf = nmf_tf.transform(tfidf.transform(data['text_2_norm']))

X_text_nmf_tf = np.concatenate([X_text_1_nmf_tf, X_text_2_nmf_tf], axis=1)

In [53]:
X_text_1_nmf = nmf.transform(cv.transform(data['text_1_norm']))
X_text_2_nmf = nmf.transform(cv.transform(data['text_2_norm']))

X_text_nmf_cv = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

In [55]:
print('cv', X_text_nmf_cv.shape)
print('tfidf', X_text_nmf_tf.shape)

cv (7227, 100)
tfidf (7227, 100)


### Word2Vec и Fastext

Преобразование текста в вектор с помощью w2v и fasttext не тривиальная задача. Самый простой и распространенный способ - усреднение отдельных векторов слов. Можно ещё использовать tfidf отдельных слов, для взвешивания отдельных векторов (чтобы частотные векторы не утягивали все на себя).

In [61]:
def get_embedding(text, model, dim, weighted=True):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            if weighted:
                weigh = tfidf.vocabulary_[word]
            else:
                weigh = (words[word]/total)
            v = model[word]
            vectors[i] = v*weigh # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
        

In [59]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim, weighted=False)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim, weighted=False)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [63]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [62]:
dim = 50
X_text_1_w2v_w = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v_w = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v_w[i] = get_embedding(text, w2v, dim, weighted=True)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v_w[i] = get_embedding(text, w2v, dim, weighted=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [64]:
X_text_w2v_w = np.concatenate([X_text_1_w2v_w, X_text_2_w2v_w], axis=1)

Fast_Text

In [65]:
#без нормализации и взвешивания
dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim, weighted=False)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim, weighted=False)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [66]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [70]:
#без взвешивания, с нормализацией
dim = 50

X1_ft_norm = np.zeros((len(data['text_1_norm']), dim))
X2_ft_norm = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X1_ft_norm[i] = get_embedding(text, fast_text, dim, weighted=False)
    
for i, text in enumerate(data['text_2_norm'].values):
    X2_ft_norm[i] = get_embedding(text, fast_text, dim, weighted=False)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [71]:
X_text_ft_norm = np.concatenate([X1_ft_norm, X2_ft_norm], axis=1)

In [73]:
#взвешенные без нормализации
dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft_w = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft_w = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft_w[i] = get_embedding(text, fast_text, dim, weighted=True)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft_w[i] = get_embedding(text, fast_text, dim, weighted=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [74]:
X_text_ft_w = np.concatenate([X_text_1_ft_w, X_text_2_ft_w], axis=1)

In [75]:
dim = 50

X1_ft_norm_w = np.zeros((len(data['text_1_norm']), dim))
X2_ft_norm_w = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X1_ft_norm_w[i] = get_embedding(text, fast_text, dim, weighted=True)
    
for i, text in enumerate(data['text_2_norm'].values):
    X2_ft_norm_w[i] = get_embedding(text, fast_text, dim, weighted=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [76]:
X_text_ft_norm_w = np.concatenate([X1_ft_norm_w, X2_ft_norm_w], axis=1)

In [77]:
vecs = [X_text_tf, X_text_cv, X_text_nmf_tf, X_text_nmf_cv, X_text_w2v, X_text_w2v_w, 
       X_text_ft, X_text_ft_norm, X_text_ft_w, X_text_ft_norm_w]

In [101]:
X_text_ft_norm_w.shape

(7227, 100)

In [78]:
from gensim import matutils


def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [93]:
dist_ = {}
for i, t in enumerate(vecs):
    text1 = t[0]
    text2 = t[1]
    fin = []  
    for j, pair in enumerate(text1):
        d = cosine_distances(text1[j].reshape(1, -1), text2[j].reshape(1, -1))[0]
        fin.append(d[0])    
    dist_[i] = fin

Дальше все сломалось

In [94]:
cos = pd.DataFrame(dist_)
cos.head()

ValueError: arrays must all be same length

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_ft, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))
